In [1]:
import gpkit
from gpkit import Model, Variable
from gpkit.constraints.tight import Tight
from gpkit.constraints.loose import Loose
from gpkit.constraints.bounded import Bounded
from gpkit import ureg

from prettytable import PrettyTable
import operator

from rocket import Rocket
from sankey3 import Sankey

import datetime

with gpkit.SignomialsEnabled():
    r = Rocket()
    prob = Model(r.m, Bounded(r))
    sol = prob.localsolve(iteration_limit=10)
    sol.datetime = datetime.datetime.now().strftime('%d-%m %H:%M')

print()
print(f'Solved at {sol.datetime}')        
print(sol.summary())

​Beginning signomial solve.
Solving took 5 GP solves and 1.6 seconds.

Solved at 18-10 14:08

Cost
----
 32.74 [kg]

Warnings
Unexpectedly Tight Constraints
------------------------------
 +0.27 : L_{ox} >= 0.5

Free Variables
--------------
               | Rocket
    a_{launch} : 90         [m/s²] Acceleration off launch rail
             m : 32.74      [kg]   Mass of Rocket
         min_a : 90         [m/s²] minimum launch acceleration

               | Rocket/Boosters
             F : 2268       [N]    Boosters cumulative thrust
             m : 1.26       [kg]   Mass of Boosters
       m_{dry} : 0.8819     [kg]   Dry mass of boosters
      m_{prop} : 0.378      [kg]   Propellant mass of boosters
      t_{burn} : 0.3333     [s]    Booster burn time

               | Rocket/SimpleEngine
     A_{grain} : 127.2      [cm²]  cross section area of grain
     L_{grain} : 0.08173    [m]    Length of the grain
        L_{ox} : 0.5        [m]    Length of ox tank
        V_{ox} : 1.207e+04  

In [2]:

d=sol['sensitivities']['variables']
variables = sol['variables']
t = PrettyTable(['key','lineage','value','unit','sens', '-abs(sens)','descr'])
for key, val in d.items():
    if key in sol['constants'].keys():
        sens =  '%4.3f'%val
    else:
        sens = '*'
    
    t.add_row([key.name,key.lineagestr(),float(variables[key]),key.unitrepr,sens,-abs(float(val)),key.label])
    
    

t.align='r'
t.float_format='5.3'
t.reversesort = False
t.sortby = '-abs(sens)'

t.sort_key=operator.itemgetter(2,6)
print('SORTED BY LINEAGE, SENSITIVITY (solved on ' + sol.datetime + ')')
print(t.get_string(fields = ['key', 'lineage','value','unit','sens','descr']))


SORTED BY LINEAGE, SENSITIVITY (solved on 18-10 14:08)
+-------------------+---------------------+-----------+--------+--------+---------------------------------------+
|               key |             lineage |     value |   unit |   sens |                                 descr |
+-------------------+---------------------+-----------+--------+--------+---------------------------------------+
|               PMF |              Rocket |     0.200 |      - |  0.325 |     Propellant Mass Fraction required |
|        v_{launch} |              Rocket |    30.000 |    m/s |  0.086 |              Velocity off launch rail |
|        L_{launch} |              Rocket |     5.000 |      m | -0.016 |                 Length of launch rail |
|                 g |              Rocket |     9.810 |  m/s^2 |  0.008 |           Acceleration due to gravity |
|        a_{launch} |              Rocket |    90.000 |  m/s^2 |      * |          Acceleration off launch rail |
|                 m |            

In [3]:
t.sort_key=operator.itemgetter(6)
print('SORTED BY SENSITIVITY (solved on ' + sol.datetime + ')')
print(t.get_string(fields = ['key', 'lineage','value','unit','sens','descr']))

SORTED BY SENSITIVITY (solved on 18-10 14:08)
+-------------------+---------------------+-----------+--------+--------+---------------------------------------+
|               key |             lineage |     value |   unit |   sens |                                 descr |
+-------------------+---------------------+-----------+--------+--------+---------------------------------------+
|              d_ox | Rocket/SimpleEngine |    18.000 |     cm |  0.548 |                   Diameter of ox tank |
|               PMF |              Rocket |     0.200 |      - |  0.325 |     Propellant Mass Fraction required |
|                SF | Rocket/SimpleEngine |     5.000 |      - |  0.319 |          Wall thickness safety factor |
|      \sigma_{max} | Rocket/SimpleEngine |   430.000 |    MPa | -0.319 |        Max stress of tank, Al-7075-T6 |
|            Tank P | Rocket/SimpleEngine |    80.000 |    bar |  0.319 |                  Max Ox Tank pressure |
|    rho_{ox, tank} | Rocket/SimpleEngine 

In [4]:
Sankey(prob).diagram(r.m)

(objective) adds +1 to the sensitivity of m_Rocket
(objective) is m_Rocket [kg] 

Ⓐ adds -1.4 to the overall sensitivity of m_Rocket
Ⓐ is m_Rocket >= m_Rocket/Avionics + m_Rocket/Boosters + m_Rocket/Payload + m_Rocket/Recovery + m_Rocket/SimpleEngine + m_Rocket/Structures 

Ⓑ adds +0.325 to the overall sensitivity of m_Rocket
Ⓑ is m_{prop}_Rocket/SimpleEngine >= PMF_Rocket*m_Rocket 

Ⓒ adds +5.9e-11 to the overall sensitivity of m_Rocket
Ⓒ is m_Rocket <= 100 

Ⓓ adds -5.37e-11 to the overall sensitivity of m_Rocket
Ⓓ is m_Rocket >= 10 

Ⓔ adds +3.63e-11 to the overall sensitivity of m_Rocket
Ⓔ is F_Rocket/SimpleEngine >= TW_{main, min}_Rocket*g_Rocket*m_Rocket 

Ⓕ adds +7.16e-13 to the overall sensitivity of m_Rocket
Ⓕ is m_Rocket <= 1e+30 

Ⓖ adds -6.54e-13 to the overall sensitivity of m_Rocket
Ⓖ is m_Rocket >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_Rocket', 'target': 'Model', 'value…

In [5]:
s = Sankey(prob)

In [6]:
s.sorted_by("constraints", 9)

Ⓐ adds +0.054 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓐ is m_Rocket/Boosters >= m_{dry}_Rocket/Boosters + m_{prop}_Rocket/Boosters 

Ⓑ adds -5.92e-10 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓑ is m_{prop}_Rocket/Boosters >= 0.2 

Ⓒ adds -0.054 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓒ is c_Rocket/Boosters*m_{prop}_Rocket/Boosters >= F_Rocket/Boosters*t_{burn}_Rocket/Boosters 

Ⓓ adds +6.49e-13 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓓ is m_{prop}_Rocket/Boosters <= 1e+30 

Ⓔ adds -7.21e-13 to the overall sensitivity of m_{prop}_Rocket/Boosters
Ⓔ is m_{prop}_Rocket/Boosters >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_{prop}_Rocket/Boosters', 'target':…

In [7]:
s.sorted_by("maxflow", 3)

Ⓐ adds +0.319 to the overall sensitivity of d_ox_Rocket/SimpleEngine
Ⓐ is t_{wall}_Rocket/SimpleEngine >= 0.5*SF_Rocket/SimpleEngine*Tank P_Rocket/SimpleEngine*\sigma_{max}_Rocket/SimpleEngine**-1*d_ox_Rocket/SimpleEngine 

Ⓑ adds -1.99e-11 to the overall sensitivity of d_ox_Rocket/SimpleEngine
Ⓑ is L_{ox}_Rocket/SimpleEngine >= 1.27*V_{ox}_Rocket/SimpleEngine*d_ox_Rocket/SimpleEngine**-2 

Ⓒ adds +0.274 to the overall sensitivity of d_ox_Rocket/SimpleEngine
Ⓒ is m_{ox tank}_Rocket/SimpleEngine >= 3.14*L_{ox}_Rocket/SimpleEngine*d_ox_Rocket/SimpleEngine*rho_{ox, tank}_Rocket/SimpleEngine*t_{wall}_Rocket/SimpleEngine 

Ⓓ adds -0.0895 to the overall sensitivity of d_ox_Rocket/SimpleEngine
Ⓓ is A_{grain}_Rocket/SimpleEngine <= 0.393*d_ox_Rocket/SimpleEngine**2 

Ⓔ adds +0.0448 to the overall sensitivity of d_ox_Rocket/SimpleEngine
Ⓔ is m_{grain tank}_Rocket/SimpleEngine >= 3.14*L_{grain}_Rocket/SimpleEngine*d_ox_Rocket/SimpleEngine*rho_{ox, tank}_Rocket/SimpleEngine*t_{wall}_Rocket/Simple

SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'd_ox_Rocket/SimpleEngine', 'target':…

In [8]:
Sankey(prob).diagram(r.engine.m_ox)

Ⓐ adds +1.98 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓐ is m_{prop}_Rocket/SimpleEngine >= m_{fuel}_Rocket/SimpleEngine + m_{ox}_Rocket/SimpleEngine 

Ⓑ adds -1.98 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓑ is OF_Rocket/SimpleEngine**-1*m_{ox}_Rocket/SimpleEngine + m_{ox}_Rocket/SimpleEngine >= m_{prop}_Rocket/SimpleEngine 

Ⓒ adds +1.01e-11 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓒ is V_{ox}_Rocket/SimpleEngine >= m_{ox}_Rocket/SimpleEngine*rho_{ox}_Rocket/SimpleEngine**-1 

Ⓓ adds +6.99e-13 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓓ is m_{ox}_Rocket/SimpleEngine <= 1e+30 

Ⓔ adds -6.71e-13 to the overall sensitivity of m_{ox}_Rocket/SimpleEngine
Ⓔ is m_{ox}_Rocket/SimpleEngine >= 1e-30 



SankeyWidget(layout=Layout(height='400', width='900'), links=[{'source': 'm_{ox}_Rocket/SimpleEngine', 'target…